In [1]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem

plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["figure.dpi"] = 300


sys.path.append("scripts")
sys.path.append("../../scripts")

In [2]:
from dim_reduction import SpectralEmbedding, scBiMapEmbedding,GaussianRandomProjection,SparseRandomProjection
from meta_nearest_neighbors import (
    ExactNearestNeighbors,
    HNSW,
    NearestNeighborsConfig,
    compute_nearest_neighbors
)

In [3]:
config_dict = {}
config_dict["HNSW_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["HNSW_Cosine_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine'))
config_dict["HNSW_Cosine_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["HNSW_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Cosine,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Euclidean_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Euclidean_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
config_dict["Exact_Cosine_Spectural_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_Spectural_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,SpectralEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=SpectralEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_100d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,100,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=100),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,TF',
            tfidf='TF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,IDF',
            tfidf='IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_scBiMap_500d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,scBiMapEmbedding,500,None',
            tfidf='None',
            dimension_reduction_method=scBiMapEmbedding,
            dimension_reduction_kw=dict(n_dimensions=500),
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,TF',
            tfidf='TF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,IDF',
            tfidf='IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,TF-IDF',
            tfidf='TF-IDF',
            nearest_neighbors_kw=dict(metric='cosine')) 
config_dict["Exact_Cosine_None_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Cosine,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='cosine')) 

#-----HNSW and random projection
config_dict["HNSW_Euclidean_GaussianRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,None',
            tfidf='None',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["HNSW_Euclidean_GaussianRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["HNSW_Euclidean_GaussianRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["HNSW_Euclidean_GaussianRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,GaussianRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 

config_dict["HNSW_Euclidean_SparseRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,None',
            tfidf='None',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["HNSW_Euclidean_SparseRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["HNSW_Euclidean_SparseRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["HNSW_Euclidean_SparseRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=HNSW,
            description='HNSW,Euclidean,SparseRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 

#-----exact and random projection

config_dict["Exact_Euclidean_GaussianRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,None',
            tfidf='None',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["Exact_Euclidean_GaussianRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["Exact_Euclidean_GaussianRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["Exact_Euclidean_GaussianRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,GaussianRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=GaussianRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 

config_dict["Exact_Euclidean_SparseRP_3200d_None"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,None',
            tfidf='None',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["Exact_Euclidean_SparseRP_3200d_TF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,TF',
            tfidf='TF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["Exact_Euclidean_SparseRP_3200d_IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,IDF',
            tfidf='IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 
config_dict["Exact_Euclidean_SparseRP_3200d_TF-IDF"]=NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='Exact,Euclidean,SparseRP,3200,TF-IDF',
            tfidf='TF-IDF',
            dimension_reduction_method=SparseRandomProjection,
            dimension_reduction_kw=dict(n_dimensions=3200),
            nearest_neighbors_kw=dict(metric='Euclidean')) 

In [4]:
import pickle
with open('/home/miaocj/docker_dir/kNN-overlap-finder/workflow/notebooks/meta_config_dict.pkl', 'wb') as file:  
    pickle.dump(config_dict, file) 

In [5]:
config_dict["Exact_Euclidean_SparseRP_3200d_TF-IDF"]

NearestNeighborsConfig(description='Exact,Euclidean,SparseRP,3200,TF-IDF', tfidf='TF-IDF', dimension_reduction_method=<class 'dim_reduction.SparseRandomProjection'>, nearest_neighbors_method=<class 'meta_nearest_neighbors.ExactNearestNeighbors'>)

In [36]:
with open('/home/miaocj/docker_dir/kNN-overlap-finder/workflow/notebooks/config_dict.pkl', 'rb') as file:  
    config_dict1 = pickle.load(file) 
config_dict1["NNDescent_Euclidean_Spectural_500d_TF-IDF"]

NearestNeighborsConfig(description='NNDescent,Euclidean,SpectralEmbedding,500,TF-IDF', tfidf='TF-IDF', dimension_reduction_method=<class 'dim_reduction.SpectralEmbedding'>, nearest_neighbors_method=<class 'nearest_neighbors.NNDescent'>)

In [42]:
mytest1 = []
for method in simple_names:
    if '100' not in method and 'scBiMap' in method:
        if 'HNSW' in method:
            mytest1.append(method)
        elif 'PQ' in method:
            mytest1.append(method)
        elif 'Exact' in method:
            mytest1.append(method)
print(mytest1)
len(mytest1)

['Exact_Euclidean_scBiMap_500d_TF', 'Exact_Euclidean_scBiMap_500d_IDF', 'Exact_Euclidean_scBiMap_500d_TF-IDF', 'Exact_Euclidean_scBiMap_500d_None', 'Exact_Cosine_scBiMap_500d_TF', 'Exact_Cosine_scBiMap_500d_IDF', 'Exact_Cosine_scBiMap_500d_TF-IDF', 'Exact_Cosine_scBiMap_500d_None', 'PQ_Euclidean_scBiMap_500d_TF', 'PQ_Euclidean_scBiMap_500d_IDF', 'PQ_Euclidean_scBiMap_500d_TF-IDF', 'PQ_Euclidean_scBiMap_500d_None', 'PQ_Cosine_scBiMap_500d_TF', 'PQ_Cosine_scBiMap_500d_IDF', 'PQ_Cosine_scBiMap_500d_TF-IDF', 'PQ_Cosine_scBiMap_500d_None', 'HNSW_Euclidean_scBiMap_500d_TF', 'HNSW_Euclidean_scBiMap_500d_IDF', 'HNSW_Euclidean_scBiMap_500d_TF-IDF', 'HNSW_Euclidean_scBiMap_500d_None', 'HNSW_Cosine_scBiMap_500d_TF', 'HNSW_Cosine_scBiMap_500d_IDF', 'HNSW_Cosine_scBiMap_500d_TF-IDF', 'HNSW_Cosine_scBiMap_500d_None']


24

In [16]:
mytest2 = []
for method in simple_names:
    if 'TF-IDF' in method:
        if '100' not in method and 'scBiMap' in method:
            if 'Exact' in method:
                mytest2.append(method)
        elif 'Min' in method or 'Low' in method:
            mytest2.append(method)
        elif 'Exact' in method and 'None_' in method:
            mytest2.append(method)
mytest2

['Exact_Euclidean_scBiMap_500d_TF-IDF',
 'Exact_Euclidean_None_TF-IDF',
 'Exact_Cosine_scBiMap_500d_TF-IDF',
 'Exact_Cosine_None_TF-IDF',
 'LowHash_Jaccard_None_TF-IDF',
 'WeightedLowHash_Jaccard_None_TF-IDF',
 'MinHash_Jaccard_None_TF-IDF',
 'WeightedMinHash_Jaccard_None_TF-IDF']

In [20]:
mytest2 = []
for method in simple_names:
    if 'TF-IDF' in method and ('500' in method or 'Hash' in method):
        mytest2.append(method)
mytest2

['Exact_Euclidean_Spectural_500d_TF-IDF',
 'Exact_Euclidean_scBiMap_500d_TF-IDF',
 'Exact_Cosine_Spectural_500d_TF-IDF',
 'Exact_Cosine_scBiMap_500d_TF-IDF',
 'NNDescent_Euclidean_Spectural_500d_TF-IDF',
 'NNDescent_Euclidean_scBiMap_500d_TF-IDF',
 'PQ_Euclidean_Spectural_500d_TF-IDF',
 'PQ_Euclidean_scBiMap_500d_TF-IDF',
 'PQ_Cosine_Spectural_500d_TF-IDF',
 'PQ_Cosine_scBiMap_500d_TF-IDF',
 'HNSW_Euclidean_Spectural_500d_TF-IDF',
 'HNSW_Euclidean_scBiMap_500d_TF-IDF',
 'HNSW_Cosine_Spectural_500d_TF-IDF',
 'HNSW_Cosine_scBiMap_500d_TF-IDF',
 'LowHash_Jaccard_None_TF-IDF',
 'WeightedLowHash_Jaccard_None_TF-IDF',
 'MinHash_Jaccard_None_TF-IDF',
 'WeightedMinHash_Jaccard_None_TF-IDF',
 'NNDescent_Euclidean_Spectural_500d_TF-IDF',
 'NNDescent_Euclidean_scBiMap_500d_TF-IDF']